# Usando Whisper para transcrever vídeo

### A ideia  é gerar através do Whisper uma legenda para um vídeo usando o modelo para transcrever o aúdio e logo após fazer a tradução e criar uma legenda (.srt) para o vídeo.

Instalando as bibliotecas necessárias

In [ ]:
!pip install openai-whisper moviepy srt googletrans==4.0.0-rc1 pysrt

Importando bibliotecas e funções

In [10]:
import whisper
import os
from googletrans import Translator
import srt
from moviepy.editor import VideoFileClip, CompositeVideoClip, TextClip
from datetime import timedelta
from moviepy.video.tools.subtitles import SubtitlesClip
import pysrt

É necessário instalar o ffmpeg como pré-requisito pra utilizar o modelo whisper, consultar: https://phoenixnap.com/kb/ffmpeg-windows

In [11]:
os.environ["PATH"] += os.pathsep + "C:/ffmpeg/bin"

Função para converter segundos em formato timedelta e posteriormente usar para gerar os times da legenda

In [12]:
def seconds_to_timedelta(seconds):
    return timedelta(seconds=int(seconds), milliseconds=int((seconds - int(seconds)) * 1000))

Extraindo áudio do vídeo com o moviepy para ser usado na transcrição

In [13]:
audio_path = "audio.mp3"

In [14]:
video_clip = VideoFileClip("linear_regression.mp4")
video_clip.audio.write_audiofile(audio_path)

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


Carrega o modelo Whisper

In [15]:
model = whisper.load_model("base")


Faz a trancrição do aúdio

In [16]:
result = model.transcribe(audio_path, language="en")

Processa a transcrição para traduzir e gerar legenda

In [18]:
# Inicializar o tradutor do googletrans
translator = Translator()

# Passo 3: Processar a transcrição para gerar legendas
subtitles = []
for i, segment in enumerate(result['segments']):
    start_time = seconds_to_timedelta(segment['start'])
    end_time = seconds_to_timedelta(segment['end'])
    # Traduzir a transcrição para o português
    translated_text = translator.translate(segment['text'], src='en', dest='pt').text
    subtitle = srt.Subtitle(index=i + 1, start=start_time, end=end_time, content=translated_text)
    subtitles.append(subtitle)


Cria uma legenda com o formato .str

In [20]:
with open("linear_regression.srt", "w", encoding="utf-8") as f:
    f.write(srt.compose(subtitles))